In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.ion()

In [3]:
with open("./dataset/resnet18/x.data", "rb") as f:
     x = np.load(f)
        
with open("./dataset/resnet18/y.data", "rb") as f:
     y = np.load(f)

In [4]:
print(x.shape, y.shape)

(40000, 512) (40000,)


In [5]:
from sklearn.svm import SVR

model = SVR(gamma='scale', epsilon=0.1)

model.fit(x, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [15]:
import torchvision.models as models
import torch.nn as nn
import torch
import datautil

feat_model = models.resnet18(pretrained=True)

# Strip the last linear layer
feat_model.fc = nn.Sequential()

pipeline = datautil.Pipeline(feat_model, model)
_, _, scaler = datautil.create_loader(None)

In [16]:
import os

# Read the diplecs dataset
diplecs_root = "../diplecs-dataset/224_dataset"
image_files = sorted(os.listdir(diplecs_root + "/images"))[:1000]

In [24]:
import cv2

In [25]:
from PIL import Image
import importlib
importlib.reload(datautil)

images = [Image.open(diplecs_root + "/images/" + p).convert("RGB") for p in image_files]
angles = scaler.inverse_transform(pipeline.predict(images).reshape(-1, 1))

torch.Size([1000, 512])


In [70]:
import math

fps = 24
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter("output.avi", fourcc, fps, images[0].size)

def draw_line(res, angle):
    start = (int(res.shape[1]/2), int(res.shape[0]/2))
    size = 50
    
    end = (
        int(start[0] + size*math.sin(-angle)),
        int(start[1] - size*math.cos(-angle))
    )
    
    color = (255, 255, 255)
    thickness = 2
    cv2.arrowedLine(res, start, end, color, thickness)

last_angles = 0
for (idx, (img, angle)) in enumerate(zip(images, angles)):
    res = np.array(img)
    if idx % 10 == 0:
        last_angle = angle

    cv2.putText(res, "%.2f" % last_angle, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 128, 128), 2, cv2.LINE_AA)
    draw_line(res, last_angle)

    out.write(res)
    
out.release()

In [41]:
images[0].size

(224, 168)

In [60]:
np.array(images[0]).shape

(168, 224, 3)

In [69]:
np.mean

<function numpy.mean(a, axis=None, dtype=None, out=None, keepdims=<no value>)>